In [56]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import datetime
import re
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import PatternFill
from copy import copy

In [57]:
try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        query = """SELECT voo_operations.department,
CASE
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 1300  
        THEN 'До 1300уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 1300 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 4000  
        THEN '1300-4000уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 4000 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 7000  
        THEN '4000-7000уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 7000 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 13000  
        THEN '7000-13000уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 13000 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 27000  
        THEN '13000-27000уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 27000 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 39500  
        THEN '27000-39500уе'
    WHEN (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) >= 39500 AND (voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) < 40000  
        THEN '39500-40000уе'		  		  		  		          
    ELSE '40000+'
END AS Category, 
	CONCAT(CAST(YEAR(voo_operations.operation_date) AS CHAR), '_', CAST(WEEK(voo_operations.operation_date, 1) AS CHAR)) AS Год_неделя,
	ROUND(SUM((voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))))) AS Оборот,
	ROUND(SUM((voo_operations.operation_revenue / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))))) AS Доход,
	COUNT(voo_operations.operation_sum) AS Кол_во_операций
	
FROM voo_operations
WHERE voo_operations.operation_date >= CAST(DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 2 YEAR) ,'%Y-01-01') as DATE)
GROUP BY 1, 2, 3
ORDER BY voo_operations.department, voo_operations.operation_date, voo_operations.operation_sum"""

        df = pd.read_sql(query, connection)

except Error as e:
    print(e)


In [102]:
df_results = df.groupby(['department', 'Год_неделя']).sum().reset_index()
df_results['Category'] = 'Итог'

df = df.append(df_results)

filials = sorted(list(df.department.unique()))

filials_dct = dict(zip(filials, range(len(filials))))

categories_dct = {'До 1300уе': 1,
'1300-4000уе': 2,
'4000-7000уе': 3,
'7000-13000уе': 4,
'13000-27000уе': 5,
'27000-39500уе': 6,
'39500-40000уе': 7,                
'40000+': 8,
'Итог': 9}

pattern = r'^\d{4}_\d{1,2}$'

def sort_func(arr):
    if all([i in filials_dct.keys() for i in arr]):
        return [filials_dct[i] for i in arr]
    elif all([i in categories_dct.keys() for i in arr]):
        return [categories_dct[i] for i in arr]
    else:
        return [tuple(map(int, i.split('_'))) for i in arr]
    
def sort_columns(arr):
    if all([i in ('Оборот', 'Доход', 'Кол_во_операций') for i in arr]):
        return [{'Оборот': 1, 'Доход': 2, 'Кол_во_операций': 3}[i] for i in arr]
    elif all([re.match(pattern, i) for i in arr]):
        dct = dict(zip(sorted(set([tuple(map(int, i.split('_'))) for i in arr])), range(len(sorted(set([tuple(map(int, i.split('_'))) for i in arr]))))))
        return [dct[tuple(map(int, i.split('_')))] for i in arr]
    
new_df = df.sort_values(by=['department', 'Category', 'Год_неделя'], key=sort_func)    

new_df = df.pivot_table(index=['department', 'Category'], columns='Год_неделя', values=['Оборот', 'Доход', 'Кол_во_операций'], aggfunc=sum).sort_index(key=sort_func)

new_df
#df.sort_index(key=sort_columns, axis=1)

Доход                        \
Год_неделя                              2019_10    2019_11    2019_12   
department             Category                                         
Головное отделение     До 1300уе         1071.0     1548.0     1827.0   
                       1300-4000уе        848.0     1327.0     1687.0   
                       4000-7000уе        462.0     1066.0     1382.0   
                       7000-13000уе       537.0     1237.0     1001.0   
                       13000-27000уе      236.0      491.0     1242.0   
...                                         ...        ...        ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе     1568.0     1734.0     1975.0   
                       27000-39500уе      376.0      739.0     1351.0   
                       39500-40000уе        NaN        NaN        NaN   
                       40000+             621.0      910.0     2562.0   
                       Итог           3361869.0  4074203.0  5807004.0   

                                                                       \
Год_неделя                              2019_13    2019_14    2019_15   
department             Category                                         
Головное отделение     До 1300уе         1500.0     1327.0     1234.0   
                       1300-4000уе       1213.0     1173.0     1240.0   
                       4000-7000уе        758.0      672.0      635.0   
                       7000-13000уе       962.0      650.0     1261.0   
                       13000-27000уе     1094.0      700.0      699.0   
...                                         ...        ...        ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе     1525.0     2303.0     1968.0   
                       27000-39500уе     2297.0     1317.0      435.0   
                       39500-40000уе        NaN      138.0      262.0   
                       40000+             886.0      756.0     3374.0   
                       Итог           5306224.0  4899979.0  5482008.0   

                                                                       \
Год_неделя                              2019_16    2019_17    2019_18   
department             Category                                         
Головное отделение     До 1300уе         1539.0     1528.0     1000.0   
                       1300-4000уе       1082.0     1413.0     1047.0   
                       4000-7000уе        857.0      639.0      550.0   
                       7000-13000уе       758.0     1098.0     1074.0   
                       13000-27000уе      532.0      470.0      245.0   
...                                         ...        ...        ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе     1931.0     2942.0     2170.0   
                       27000-39500уе     1130.0     1710.0      126.0   
                       39500-40000уе        NaN      157.0      502.0   
                       40000+            1209.0     3834.0     1000.0   
                       Итог           4663386.0  7373730.0  3744608.0   

                                                 ...        Оборот  \
Год_неделя                              2019_19  ...       2021_19   
department             Category                  ...                 
Головное отделение     До 1300уе          992.0  ...  1.907990e+05   
                       1300-4000уе        653.0  ...  2.140500e+05   
                       4000-7000уе        543.0  ...  2.209200e+05   
                       7000-13000уе       387.0  ...  2.322280e+05   
                       13000-27000уе      396.0  ...  1.396940e+05   
...                                         ...  ...           ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе      856.0  ...  6.701010e+05   
                       27000-39500уе      428.0  ...  3.373900e+05   
                       39500-40000уе        NaN  ...  2.398000e+05   
                       40000+            1526.0  ...  7.222000e+05   
                       Итог           3203459.0  ...  1.805877e

In [76]:
print(len(new_df.columns))




372


In [88]:
x = new_df.sort_index(key=sort_columns, axis=1, level=0)#.sort_index(key=sort_columns, axis=1, level=1)

In [103]:
x = new_df.sort_index(key=sort_columns, axis=1)

In [104]:
x

Оборот                \
Год_неделя                                  2019_2        2019_3   
department             Category                                    
Головное отделение     До 1300уе      2.446370e+05  3.099080e+05   
                       1300-4000уе    2.208210e+05  3.336760e+05   
                       4000-7000уе    1.091850e+05  2.380760e+05   
                       7000-13000уе   1.886160e+05  2.695100e+05   
                       13000-27000уе  1.518200e+05  1.822750e+05   
...                                            ...           ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе  2.861400e+05  4.353000e+05   
                       27000-39500уе  2.069600e+05  1.568500e+05   
                       39500-40000уе  4.000000e+04           NaN   
                       40000+         1.059360e+06  7.464820e+05   
                       Итог           1.194194e+09  1.244342e+09   

                                                                  \
Год_неделя                                  2019_4        2019_5   
department             Category                                    
Головное отделение     До 1300уе      3.174740e+05  3.563940e+05   
                       1300-4000уе    3.037820e+05  4.020330e+05   
                       4000-7000уе    1.750150e+05  2.492100e+05   
                       7000-13000уе   2.650500e+05  2.546000e+05   
                       13000-27000уе  2.620260e+05  2.197200e+05   
...                                            ...           ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе  5.702630e+05  5.597650e+05   
                       27000-39500уе  3.485000e+05  2.413500e+05   
                       39500-40000уе           NaN  1.196000e+05   
                       40000+         6.109220e+05  8.556200e+05   
                       Итог           1.378710e+09  1.556543e+09   

                                                                               \
Год_неделя                                  2019_6        2019_7       2019_8   
department             Category                                                 
Головное отделение     До 1300уе      2.917710e+05  3.323490e+05     326579.0   
                       1300-4000уе    3.081400e+05  3.610710e+05     314595.0   
                       4000-7000уе    1.658100e+05  2.018380e+05     242654.0   
                       7000-13000уе   3.014500e+05  1.935130e+05     280267.0   
                       13000-27000уе  9.935000e+04  2.517200e+05     296301.0   
...                                            ...           ...          ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе  4.555210e+05  5.007000e+05     319050.0   
                       27000-39500уе  2.592700e+05  1.870000e+05     197280.0   
                       39500-40000уе  3.970000e+04           NaN          NaN   
                       40000+         6.259200e+05  6.968500e+05      94000.0   
                       Итог           1.200336e+09  1.254717e+09  878117730.0   

                                                                               \
Год_неделя                                  2019_9      2019_10       2019_11   
department             Category                                                 
Головное отделение     До 1300уе      3.619810e+05     290361.0  3.970160e+05   
                       1300-4000уе    3.472040e+05     217370.0  3.391600e+05   
                       4000-7000уе    2.436850e+05     117375.0  2.941750e+05   
                       7000-13000уе   2.758670e+05     155502.0  3.236850e+05   
                       13000-27000уе  2.478000e+05      63300.0  2.155500e+05   
...                                            ...          ...           ...   
ОКВКУ № 9 "Крылатское" 13000-27000уе  5.087500e+05     433700.0  4.859000e+05   
                       27000-39500уе  2.404500e+05      98200.0  2.128200e+05   
                       39500-40000уе           NaN          NaN           NaN   
                       40000+  

In [101]:
dict(zip(sorted(set([tuple(map(int, i[1].split('_'))) for i in new_df.columns])), range(len(sorted(set([tuple(map(int, i[1].split('_'))) for i in new_df.columns]))))))

{(2019, 2): 0,
 (2019, 3): 1,
 (2019, 4): 2,
 (2019, 5): 3,
 (2019, 6): 4,
 (2019, 7): 5,
 (2019, 8): 6,
 (2019, 9): 7,
 (2019, 10): 8,
 (2019, 11): 9,
 (2019, 12): 10,
 (2019, 13): 11,
 (2019, 14): 12,
 (2019, 15): 13,
 (2019, 16): 14,
 (2019, 17): 15,
 (2019, 18): 16,
 (2019, 19): 17,
 (2019, 20): 18,
 (2019, 21): 19,
 (2019, 22): 20,
 (2019, 23): 21,
 (2019, 24): 22,
 (2019, 25): 23,
 (2019, 26): 24,
 (2019, 27): 25,
 (2019, 28): 26,
 (2019, 29): 27,
 (2019, 30): 28,
 (2019, 31): 29,
 (2019, 32): 30,
 (2019, 33): 31,
 (2019, 34): 32,
 (2019, 35): 33,
 (2019, 36): 34,
 (2019, 37): 35,
 (2019, 38): 36,
 (2019, 39): 37,
 (2019, 40): 38,
 (2019, 41): 39,
 (2019, 42): 40,
 (2019, 43): 41,
 (2019, 44): 42,
 (2019, 45): 43,
 (2019, 46): 44,
 (2019, 47): 45,
 (2019, 48): 46,
 (2019, 49): 47,
 (2019, 50): 48,
 (2019, 51): 49,
 (2019, 52): 50,
 (2019, 53): 51,
 (2020, 2): 52,
 (2020, 3): 53,
 (2020, 4): 54,
 (2020, 5): 55,
 (2020, 6): 56,
 (2020, 7): 57,
 (2020, 8): 58,
 (2020, 9): 59,
 (2020